In [1]:
#Split location-only results by MA and cone angle.

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import glob

from XMA_finder import XMA_finder
from histo_plot import histo_plot
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

In [3]:
#open OMNI *average* CSVs
list_all = []

path = "/Users/apx059/Documents/Location_Only_Checks/OMNI Averages/**"

for path in glob.glob(path, recursive=True):
    list_all.append(path)
    
#list with only files, not folders
om_csvs = []

for element in list_all:
    if '.csv' in element:
        om_csvs.append(element)
        
om_dfs = []

for element in om_csvs:
    om = pd.read_csv(element)
    om_dfs.append(om)
    
omni_all = pd.concat(om_dfs)
omni_all['datetime'] = pd.to_datetime(omni_all['datetime'])

omni_all = omni_all.set_index('datetime')

for element in om_dfs:
    element['datetime'] = pd.to_datetime(element['datetime'])
    element = element.set_index('datetime', inplace = True)


In [4]:
##load Cluster CSVs

list_all = []

path = "/Users/apx059/Documents/23YrLocApocrita/Cluster_GIPM_SSC/**"

for path in glob.glob(path, recursive=True):
    list_all.append(path)
    
#list with only files, not folders
cl_file_list = []

for element in list_all:
    if '.csv' in element:
        cl_file_list.append(element)

cl_dfs = []

for file in cl_file_list:
    df = pd.read_csv(file,encoding='utf-8', names = ['datetime', 'GIPM X (RE)', 'GIPM Y (RE)', 'GIPM Z (RE)'])
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace = True)
    cl_dfs.append(df)

In [5]:
#add cone angle and MA to each row of one cluster df

ma_list = []
cone_a_list = []

#cl_1 = cl_dfs[0]

#ind_list = cl_1.index

#obs = ind_list[2]

#if obs in omni_all.index:
#    print('yes!')
#else:
#    print('no :( ')
    
for i in cl_dfs[0].index:
    if i in omni_all.index:
        ma_temp = omni_all['M_A'].loc[i]
        cone_a_temp = omni_all['cone angle'].loc[i]
        ma_list.append(ma_temp)
        cone_a_list.append(cone_a_temp)
    else:
        ma_list.append(np.nan)
        cone_a_list.append(np.nan)

cl_dfs[0]['MA'] = ma_list
cl_dfs[0]['cone angle'] = cone_a_list

print(cl_dfs[0])

                     GIPM X (RE)  GIPM Y (RE)  GIPM Z (RE)        MA  \
datetime                                                               
2015-02-01 00:00:30     3.937567     1.645362     6.800982  8.966667   
2015-02-01 00:01:30     3.900161     1.624216     6.792879  8.914286   
2015-02-01 00:02:30     3.861882     1.558532     6.795769  8.862500   
2015-02-01 00:03:30     3.824740     1.502867     6.798530  8.844444   
2015-02-01 00:04:30     3.792186     1.429821     6.813162  8.870000   
...                          ...          ...          ...       ...   
2016-01-31 23:55:30     7.599333     3.903911    13.402913  6.325000   
2016-01-31 23:56:30     7.602075     3.614946    13.553867  6.414286   
2016-01-31 23:57:30     7.596190     3.162821    13.706110  6.466667   
2016-01-31 23:58:30     7.583588     2.593628    13.880956  6.580000   
2016-01-31 23:59:30     7.618295     2.281361    13.993153  6.725000   

                     cone angle  
datetime                     

In [6]:
#make new dataframes split by 1. cone angle and 2. MA

df_radial = cl_dfs[0].loc[(cl_dfs[0]['cone angle'] < 30) & (cl_dfs[0]['GIPM X (RE)'].notnull())]
df_spiral = cl_dfs[0].loc[(cl_dfs[0]['cone angle'] > 30) & (cl_dfs[0]['cone angle'] < 60) & (cl_dfs[0]['GIPM X (RE)'].notnull())]
df_perpendicular = cl_dfs[0].loc[(cl_dfs[0]['cone angle'] > 60) & (cl_dfs[0]['GIPM X (RE)'].notnull())]

#now break up by MA. MA ranges <5, 5-8, 8-11, 11-14, 14-18, 18-22, >22

#radial
df_rad_less5 = df_radial.loc[(df_radial['MA'] < 5)]
df_rad_5_8 = df_radial.loc[((df_radial['MA'] >= 5) & df_radial['MA'] < 8)]
df_rad_8_11 = df_radial.loc[(df_radial['MA'] >= 8) & (df_radial['MA'] < 11)]
df_rad_11_14 = df_radial.loc[(df_radial['MA'] >= 11) & (df_radial['MA'] < 14)]
df_rad_14_18 = df_radial.loc[(df_radial['MA'] >= 14) & (df_radial['MA'] < 18)]
df_rad_18_22 = df_radial.loc[(df_radial['MA'] >= 18) & (df_radial['MA'] < 22)]
df_rad_22more = df_radial.loc[(df_radial['MA'] >= 22)]

#spiral 

df_spir_less5 = df_spiral.loc[(df_spiral['MA'] < 5)]
df_spir_5_8 = df_spiral.loc[((df_spiral['MA'] >= 5) & df_spiral['MA'] < 8)]
df_spir_8_11 = df_spiral.loc[(df_spiral['MA'] >= 8) & (df_spiral['MA'] < 11)]
df_spir_11_14 = df_spiral.loc[(df_spiral['MA'] >= 11) & (df_spiral['MA'] < 14)]
df_spir_14_18 = df_spiral.loc[(df_spiral['MA'] >= 14) & (df_spiral['MA'] < 18)]
df_spir_18_22 = df_spiral.loc[(df_spiral['MA'] >= 18) & (df_spiral['MA'] < 22)]
df_spir_22more = df_spiral.loc[(df_spiral['MA'] >= 22)]

#perpendicular

df_perp_less5 = df_perpendicular.loc[(df_perpendicular['MA'] < 5)]
df_perp_5_8 = df_perpendicular.loc[((df_perpendicular['MA'] >= 5) & df_perpendicular'MA'] < 8)]
df_perp_8_11 = df_perpendicular.loc[(df_perpendicular['MA'] >= 8) & (df_perpendicular['MA'] < 11)]
df_perp_11_14 = df_perpendicular.loc[(df_perpendicular['MA'] >= 11) & (df_perpendicular'MA'] < 14)]
df_perp_14_18 = df_perpendicular.loc[(df_perpendicular['MA'] >= 14) & (df_perpendicular['MA'] < 18)]
df_perp_18_22 = df_perpendicular.loc[(df_perpendicular['MA'] >= 18) & (df_perpendicular['MA'] < 22)]
df_perp_22more = df_perpendicular.loc[(df_perpendicular['MA'] >= 22)]

In [ ]:
cl_all = pd.concat

In [ ]:
##use numpy histogram to get actual bin numbers
x_bin_edges = range(30)
y_bin_edges = range(-30,30)
HistXY, xedg, yedg = np.histogram2d(x_locs, y_locs, bins=[x_bin_edges, y_bin_edges])
HistXY = HistXY.T

z_bin_edges = range(-30,30)
HistXZ, xedg, zedg = np.histogram2d(x_locs, z_locs, bins=[x_bin_edges, z_bin_edges])
HistXZ = HistXZ.T

HistXY[HistXY == 0] = np.nan
HistXZ[HistXZ == 0] = np.nan

##use numpy histogram to get actual bin numbers
x_bin_edges = np.arange(0.0, 30.0, 0.5)
y_bin_edges = np.arange(-30.0, 30.0, 0.5)
HistXY_05, xedg, yedg = np.histogram2d(x_locs, y_locs, bins=[x_bin_edges, y_bin_edges])
HistXY_05 = HistXY_05.T

z_bin_edges = np.arange(-30.0,30.0, 0.5)
HistXZ_05, xedg, zedg = np.histogram2d(x_locs, z_locs, bins=[x_bin_edges, z_bin_edges])
HistXZ_05 = HistXZ_05.T

HistXY_05[HistXY_05 == 0] = np.nan
HistXZ_05[HistXZ_05 == 0] = np.nan

XMA_all = XMA_finder(omni_all)


In [ ]:
#import modules
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib import cm
import matplotlib
from merka05_surface_eq_array_GIPM import merka05_surface_eq_array_GIPM

x = np.linspace(0, 20, 100) #x coordinates (Re)
y = np.linspace(-30, 30, 100) #y coordinates (Re)
z = 0 #z coordinates in Re

[Xgipm,Ygipm,Zgipm] = np.meshgrid(x,y,z,indexing="ij")

fitting_coeffs = merka05_surface_eq_array_GIPM(XMA_all)

Xn = Xgipm
Yn = Ygipm
Zn = Zgipm
f = fitting_coeffs[0]*Xn**2 + fitting_coeffs[1]*Yn**2 + fitting_coeffs[2]*Zn**2+ 2*fitting_coeffs[3]*Xn*Yn + 2*fitting_coeffs[4]*Yn*Zn + 2*fitting_coeffs[5]*Xn*Zn + 2*fitting_coeffs[6]*Xn+2*fitting_coeffs[7]*Yn + 2*fitting_coeffs[8]*Zn + fitting_coeffs[9]

#tanfieldangle = y/x

tan_angle_0 = np.tan(np.deg2rad(15))
tan_angle_1 = np.tan(np.deg2rad(45))
tan_angle_2 = np.tan(np.deg2rad(75))

#magnetopause model, D = 2 nPa

m_1 = 10.22
m_2 = 1.29
m_3 = 0.184
m_4 = 8.14
m_5 = 6.6
m_6 = 0.58
m_7 = -0.007
m_8 = 0.024

#use B_z =0
B_z = 0
D_p = 2

alpha = (m_6 + m_7*B_z)*(1 +m_8*(np.log(D_p)))

tanh_angle = m_3*(B_z+m_4)
r_0 = (m_1 + m_2*np.tanh(tanh_angle))*(D_p**(-1/m_5))

pi = np.pi

theta = np.arange(-pi/2, pi/2, 0.01)

r_mod = (2/(1+np.cos(theta)))**alpha
r = r_0*r_mod

X_shue = r*(np.cos(theta))
R_shue = r*(np.sin(theta))


###################
fig, ax = plt.subplots()
#subfigs = fig.subfigures(1, 1)
#axsLeft = subfigs.subplots(1, 3, sharey=False)
#gs = gridspec.GridSpec(2, 2)
plt.suptitle('Cluster Locations All 23 Years')

ax.contour(Xgipm[:,:,0],Ygipm[:,:,0],f[:,:,0],levels = [0],colors="black",linewidths=1)
ax.plot(X_shue, R_shue, linewidth=1, color='k')

ax.set_aspect('equal')
ax.set_xlabel("$X_\\mathrm{GIPM}$ ($R_\\mathrm{E}$)")
ax.set_ylabel("$Y_\\mathrm{GIPM}$ ($R_\\mathrm{E}$)")


#want to find three points on the bow shock surface, at y=0 and y=±8 and draw lines from there
#without exceeding current bounds of plot
inter_med = fitting_coeffs[6]**2 - (fitting_coeffs[0]*fitting_coeffs[9])
X_BS_nose = (-fitting_coeffs[6] + np.sqrt(inter_med))/fitting_coeffs[0]

x_s = X_BS_nose
y_s = 0
x_e = 30
y_e = x_e*(-tan_angle_0)

#want to also have line for just solar wind flow along y=0

ax.hlines(y=0, xmin= 0, xmax=30, linewidth=1, color='k')
#ax.plot([x_s, x_e], [y_s, y_e], color='k',linewidth=1)
cmap = matplotlib.colormaps.get_cmap('Blues') 
cmap.set_bad(color='lightgrey')
plt.imshow(HistXY, interpolation='nearest', origin='lower', extent=[xedg[0], xedg[-1], yedg[0], yedg[-1]],cmap = cmap)
#ax.set_ylim(-30,30)
#ax.set_xlim(0,30)
ax.invert_xaxis()
ax.invert_yaxis()
plt.colorbar()

plt.show()